<a href="https://colab.research.google.com/github/Chuck2Win/attention/blob/5_11/attention_follow_up_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from gensim.models import Word2Vec
import nltk
import os
from google.colab import files
from google.colab import drive
from matplotlib import pyplot as plt
import json
# google drive와 연동
drive.mount('/content/gdrive')
print(os.getcwd())
# os.chdir('..') ..은 상위 폴더로 이동
os.chdir('./gdrive/My Drive/attention_from_master_')
print(os.getcwd())


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/attention_from_master_


In [47]:
from google.colab import files
files.upload()

Saving ratings_test.txt to ratings_test.txt
Saving ratings_train.txt to ratings_train.txt


In [2]:
! pip install sentencepiece
import sentencepiece as spm
# testing
vocab_file = "./kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
inputs=[]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(line)
  print(pieces)
  print(ids)
  print() # 단어를 subword 단위로 잘 쪼개는것을 확인할 수 있습니다.

     |████████████████████████████████| 1.0MB 9.6MB/s 
겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3099, 3582, 604, 3596, 684, 4009, 3593, 109, 4189, 206, 3951, 3754, 3584]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[3228, 86, 3961, 3917, 3598, 264, 669, 1940, 3658, 777, 1442, 3788, 3754, 4243]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3099, 3585, 208, 3599, 53, 3826, 3590, 3608, 3594, 235, 3865, 3597, 61, 3817, 448, 3676, 3754, 3584]



In [3]:
print(len(vocab)) # 8007
print(len(ids))

8007
18


In [4]:
# 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0)을 추가 해 줌
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
print(inputs)
print(inputs.shape) # 3, 18 -> 가장 긴 문장을 기준으로 padding 해준다.

tensor([[3099, 3582,  604, 3596,  684, 4009, 3593,  109, 4189,  206, 3951, 3754,
         3584,    0,    0,    0,    0,    0],
        [3228,   86, 3961, 3917, 3598,  264,  669, 1940, 3658,  777, 1442, 3788,
         3754, 4243,    0,    0,    0,    0],
        [3099, 3585,  208, 3599,   53, 3826, 3590, 3608, 3594,  235, 3865, 3597,
           61, 3817,  448, 3676, 3754, 3584]])
torch.Size([3, 18])


In [0]:
# positional encoding을 해줘야한다.
def positional_encoding(seq_len,embedding_dim):
    # input shape : [n,seq_len]
    #output=torch.zeros((seq_len+1,embedding_dim),device=device)
    output=torch.zeros((seq_len,embedding_dim),device=device)
    def for_insert(output):
        m,n=output.shape
        for i in range(m):
            for j in range(n):
                output[i,j]=i/10000**(2*j/embedding_dim)
        return output
    pe=for_insert(output)
    pe[:,0::2]=torch.sin(pe[:,0::2])
    pe[:,1::2]=torch.cos(pe[:,1::2])
    # pe[0,:]=0. 없애고 진행해보자
    return pe

In [7]:
device='cpu'
positional_encoding(64,300)

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.8943e-01,  7.7356e-01,  ...,  1.0000e+00,
          1.1307e-08,  1.0000e+00],
        [ 9.0930e-01, -3.0515e-01,  9.8045e-01,  ...,  1.0000e+00,
          2.2613e-08,  1.0000e+00],
        ...,
        [-9.6612e-01,  6.8334e-01, -5.1718e-01,  ...,  1.0000e+00,
          6.8970e-07,  1.0000e+00],
        [-7.3918e-01, -1.8701e-01, -9.8982e-01,  ...,  1.0000e+00,
          7.0101e-07,  1.0000e+00],
        [ 1.6736e-01, -9.0380e-01, -7.3737e-01,  ...,  1.0000e+00,
          7.1232e-07,  1.0000e+00]])

In [0]:
# 이를 토대로 positional embedding을 구한다.
'''
위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. 
학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.
입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.
input값 중 pad(0)값을 찾습니다.
positions값중 pad부분은 0으로 변경 합니다. (mask를 활용)
positions값에 해당하는 embedding값을 구합니다
'''

'\n위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. \n학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.\n입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.\ninput값 중 pad(0)값을 찾습니다.\npositions값중 pad부분은 0으로 변경 합니다. (mask를 활용)\npositions값에 해당하는 embedding값을 구합니다\n'

'\n위에서 구해진 position encoding 값을 이용해 position embedding을 생성합니다. \n학습되는 값이 아니므로 freeze옵션을 True로 설정 합니다.\n입력 inputs과 동일한 크기를 갖는 positions값을 구합니다.\ninput값 중 pad(0)값을 찾습니다.\npositions값중 pad부분은 0으로 변경 합니다. (mask를 활용)\npositions값에 해당하는 embedding값을 구합니다\n'

In [9]:
############################# input embedding ##########################################
n_vocab=len(vocab)
embedding=128
seq_len=64 # 아싸리 크게 하면 되겠네. 딱 타이트하게 하지 말고...
Embedding=nn.Embedding(n_vocab,embedding,padding_idx=0)
Embedding.weight.required_grad=False
inputs_embedding=Embedding(inputs) 
############################# positional embedding ##########################################
pos_encoding=positional_encoding(seq_len,embedding)
positional_embedding=nn.Embedding.from_pretrained(pos_encoding,freeze=True)
positions=torch.arange(inputs.size(1)).expand(inputs.size(0),inputs.size(1))+1 # inputs size : n,seq_len # 0부터 시작해서 1을 더한 것이다.
print(positions)
pos_mask=inputs.eq(0) # mask shape : n,seq_len : 3, 18
positions.masked_fill_(pos_mask,0)  # (어차피 0에는 무엇을 곱해도 0이 되니깐)->아니지. 그리고 1을 더했다.. 그전에
print(positions)
pos_embedding=positional_embedding(positions)
print(pos_embedding[1]) # 보게 되면 0에 해당하는 부분은 0,1,0,1로 반복되고 있다.

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])
tensor([[ 8.4147e-01,  6.4791e-01,  6.8156e-01,  ...,  1.0000e+00,
          1.3335e-08,  1.0000e+00],
        [ 9.0930e-01, -1.6044e-01,  9.9748e-01,  ...,  1.0000e+00,
          2.6670e-08,  1.0000e+00],
        [ 1.4112e-01, -8.5580e-01,  7.7827e-01,  ...,  1.0000e+00,
          4.0006e-08,  1.0000e+00],
        ...,
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.00

In [10]:
# positional embedding과 token embedding을 더해야지
input_sums=inputs_embedding+pos_embedding 
# anyway, padding이 된 부분에 해당되는 부분이 0 값이 된 것은 아니다.. -> 그렇기 때문에...
inputs[0]
input_sums[0]

tensor([[-1.1057, -0.2500, -1.1481,  ...,  3.0691,  0.6567,  1.3464],
        [ 2.7157, -0.1901,  1.6571,  ..., -0.9450,  0.7493, -0.4445],
        [-1.1990, -0.3164,  0.7477,  ...,  0.7196, -1.5468,  0.2722],
        ...,
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000]],
       grad_fn=<SelectBackward>)

In [0]:
Q=input_sums # shape : 3, 18, 128
K=input_sums # inputs shape : 3, 18
V=input_sums
# inputs.shape : batch_size, seq_len
attention_mask=inputs.eq(0).unsqueeze(1).expand(Q.size(0),Q.size(1),K.size(1)) # shape : 3, 18, 18
# print(get_attention_pad_mask(Q,K))
# print(attention_mask[0].shape)
# print(attention_mask[0])

In [0]:
class scaled_dot_product_attention(nn.Module): # 고민 요소
    # 나의 초안에서 attention 함수를 차용해 옴
    def __init__(self):
        super().__init__()
               
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        scores=Q.matmul(K.transpose(-2,-1))/torch.sqrt(torch.tensor(Q.shape[-1]).float())
        # 여기에서 attention mask는 padding이 된 부분을 0으로 처리한 것
        # attention mask shape : n,seq_len,seq_len
        # 00011
        # 00011
        # 00011
        # 00011
        # 00011 
        # 요론 모양
        scores.masked_fill_(attention_mask,-1e-9)
        weight=F.softmax(scores,dim=-1)
        # weight : n, seq_len, seq_len
        attention_score=weight.matmul(V)
        # attention score : n,seq_len,dv
        return attention_score, weight

In [0]:
# 나는 multi head attention을 안했었는데, 병렬로 처리하기 위해선 더 좋으니깐
# 쩐다...★
class multi_head_attention(nn.Module): # 고민 요소
    # 나의 초안에서 attention 함수를 차용해 옴
    def __init__(self,batch_size,n_head,d_head,d_k):
        super().__init__()
        self.batch_size=batch_size
        self.n_head=n_head
        self.d_head=d_head
        self.d_k=d_k
        self.W_Q=nn.Linear(d_k,n_head*d_head)
        self.W_K=nn.Linear(d_k,n_head*d_head)
        self.W_V=nn.Linear(d_k,n_head*d_head)
        self.W_0=nn.Linear(n_head*d_head,n_head*d_head) # n_head*d_head = embedding
        self.sdpa=scaled_dot_product_attention()
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        # attention_mask shape : [n,seq_len,seq_len]
        qs=self.W_Q(Q).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2) # ★
        ks=self.W_K(K).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        vs=self.W_V(V).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        attention_mask=attention_mask.unsqueeze(1).repeat(1,self.n_head,1,1) # attention_mask shape : [n,self.n_head,seq_len,seq_len] <- mask도 동일하게 multi head로 확장시켜 나가야지
        context,weight=self.sdpa(qs,ks,vs,attention_mask) # context shape : n, n_head, seq_len, d_head
        context=context.transpose(1,2).reshape(self.batch_size,-1,self.n_head*self.d_head)
        output=self.W_0(context)
        return output, weight

In [0]:
mm=multi_head_attention(3,2,64,128)
mm.forward(Q,K,V,attention_mask)[0].shape

torch.Size([3, 18, 128])

torch.Size([3, 18, 128])

In [0]:
# masked multi_head attention
def get_attention_decoder_mask(seq):
    # seq : batch_size, query_seq_len
    subsequent_mask=torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0),seq.size(1),seq.size(1))
    # mask : batch_size, seq_len, seq_len
    mask=subsequent_mask.triu(diagonal=1)
    return mask   

In [0]:
attn_dec_mask=get_attention_decoder_mask(inputs)
attn_dec_mask.shape
attn_dec_mask[0]

tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [14]:
attn_pad_mask=inputs.eq(0).unsqueeze(1).expand(Q.size(0),Q.size(1),K.size(1))
print(attn_pad_mask[1])

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True, 

In [20]:
# pad mask와 decoder mask를 합쳐야 한다.
attention_decoder_mask=get_attention_decoder_mask(inputs)
attn_mask=torch.gt((attn_pad_mask+attention_decoder_mask),0) # element wise로 0보다 크면 True

print(attn_mask.long())

tensor([[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 

In [0]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_hidn):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=d_hidn, out_channels=d_hidn * 4, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_hidn * 4, out_channels=d_hidn, kernel_size=1)
        self.active = F.gelu

    def forward(self, inputs):
        # (batch size, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        # (bs, n_seq, d_hidn)
        return output

In [0]:
# ? torch.gt()
attn_mask=torch.gt((attention_pad_mask+attention_decoder_mask),0) # torch.gt(input,other)=> input>other인 경우 element wise
print(attn_mask[0].long())

In [0]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [0]:
config=Config({"n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidden": 256,
    "padding_idx": 0,
    "d_feedforward": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})

In [73]:
config['dropout']

0.1

In [0]:
# transformer 2차
class scaled_dot_product_attention(nn.Module): # 고민 요소
    def __init__(self,config):
        super().__init__()
        self.dropout=nn.Dropout(config['dropout'])      
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        scores=Q.matmul(K.transpose(-2,-1))/torch.sqrt(torch.tensor(Q.shape[-1]).float())
        scores.masked_fill_(attention_mask,-1e-9)
        weight=self.dropout(F.softmax(scores,dim=-1))

        # weight : n, seq_len, seq_len
        attention_score=weight.matmul(V)
        # attention score : n,seq_len,dv
        return attention_score, weight

In [0]:
class multi_head_attention(nn.Module): # 고민 요소
    # 나의 초안에서 attention 함수를 차용해 옴
    def __init__(self,config):
        
        super().__init__()
        
        self.n_head=config['n_head']
        self.d_head=config['d_head']
        self.d_k=config['d_hidden']
        self.W_Q=nn.Linear(self.d_k,self.n_head*self.d_head)
        self.W_K=nn.Linear(self.d_k,self.n_head*self.d_head)
        self.W_V=nn.Linear(self.d_k,self.n_head*self.d_head)
        self.W_0=nn.Linear(self.n_head*self.d_head,self.n_head*self.d_head) # n_head*d_head = embedding
        self.sdpa=scaled_dot_product_attention(config)
        self.dropout=nn.Dropout(config['dropout'])
    def forward(self,Q,K,V,attention_mask):
        # Q,K - shape [n,seq_len,dk] // V - shape [n,seq_len,dv]
        # self attention 이므로 dk=dv(=embedding)
        # attention_mask shape : [n,seq_len,seq_len]
        self.batch_size=Q.size(0)
        qs=self.W_Q(Q).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2) # ★
        ks=self.W_K(K).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        vs=self.W_V(V).reshape(self.batch_size,-1,self.n_head,self.d_head).transpose(1,2)
        attention_mask=attention_mask.unsqueeze(1).repeat(1,self.n_head,1,1) # attention_mask shape : [n,self.n_head,seq_len,seq_len] <- mask도 동일하게 multi head로 확장시켜 나가야지
        context,weight=self.sdpa(qs,ks,vs,attention_mask) # context shape : n, n_head, seq_len, d_head
        context=context.transpose(1,2).reshape(self.batch_size,-1,self.n_head*self.d_head)
        output=self.W_0(context)
        return output, weight

In [0]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.d_hidn=config['d_hidden']
        self.d_ff=config['d_feedforward']
        self.conv1 = nn.Conv1d(in_channels=self.d_hidn, out_channels=self.d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.d_ff, out_channels=self.d_hidn, kernel_size=1)
        self.active = F.gelu

    def forward(self, inputs):
        # (batch size, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        # (bs, n_seq, d_hidn)
        return output

In [0]:
class EncoderLayer(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config=config
        self.self_attn=multi_head_attention(config)
        self.d_hidn=config['d_hidden']
        self.layer_norm1=nn.LayerNorm(self.d_hidn,eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs) # resnet
        # (bs, n_enc_seq, d_hidn)
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs) # resnet
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        return ffn_outputs, attn_prob

In [0]:
# positional encoding을 해줘야한다.
def get_sinusoid_encoding_table(seq_len,embedding_dim):
    # input shape : [n,seq_len]
    #output=torch.zeros((seq_len+1,embedding_dim),device=device)
    output=torch.zeros((seq_len,embedding_dim),device=device)
    def for_insert(output):
        m,n=output.shape
        for i in range(m):
            for j in range(n):
                output[i,j]=i/10000**(2*j/embedding_dim)
        return output
    pe=for_insert(output)
    pe[:,0::2]=torch.sin(pe[:,0::2])
    pe[:,1::2]=torch.cos(pe[:,1::2])
    # pe[0,:]=0. 없애고 진행해보자
    return pe

In [0]:
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidden)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_enc_seq + 1, self.config.d_hidden))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)
        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config.n_layer)])
        # Module list는 connect되어 있지 않다.
    def forward(self, inputs):
        positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        # (bs, n_enc_seq, d_hidn)
        outputs = self.enc_emb(inputs) + self.pos_emb(positions)

        # (bs, n_enc_seq, n_enc_seq)
        attn_mask = inputs.eq(self.config.i_pad).unsqueeze(1).expand(inputs.size(0),inputs.size(1),inputs.size(1))

        attn_probs = []
        for layer in self.layers:
            # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
            outputs, attn_prob = layer(outputs, attn_mask)  # sequential로 해도 될 것 같은데
            attn_probs.append(attn_prob)
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return outputs, attn_probs

In [0]:
class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.self_attn = multi_head_attention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidden, eps=self.config.layer_norm_epsilon)
        self.dec_enc_attn = multi_head_attention(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidden, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config.d_hidden, eps=self.config.layer_norm_epsilon)
    
    def forward(self, dec_inputs, enc_outputs, self_attn_mask, dec_enc_attn_mask):
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq)
        self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_enc_seq)
        dec_enc_att_outputs, dec_enc_attn_prob = self.dec_enc_attn(self_att_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_enc_att_outputs = self.layer_norm2(self_att_outputs + dec_enc_att_outputs)
        # (bs, n_dec_seq, d_hidn)
        ffn_outputs = self.pos_ffn(dec_enc_att_outputs)
        ffn_outputs = self.layer_norm3(dec_enc_att_outputs + ffn_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq), (bs, n_head, n_dec_seq, n_enc_seq)
        return ffn_outputs, self_attn_prob, dec_enc_attn_prob

In [0]:
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dec_emb = nn.Embedding(self.config.n_dec_vocab, self.config.d_hidden)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_dec_seq + 1, self.config.d_hidden))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)
        self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        positions = torch.arange(dec_inputs.size(1), device=dec_inputs.device, dtype=dec_inputs.dtype).expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
        pos_mask = dec_inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)
    
        # (bs, n_dec_seq, d_hidn)
        dec_outputs = self.dec_emb(dec_inputs) + self.pos_emb(positions)

        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, self.config.i_pad)
        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs)
        # (bs, n_dec_seq, n_dec_seq)
        dec_self_attn_mask = torch.gt((dec_attn_pad_mask + dec_attn_decoder_mask), 0)
        # (bs, n_dec_seq, n_enc_seq)
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs, self.config.i_pad)

        self_attn_probs, dec_enc_attn_probs = [], []
        for layer in self.layers:
            # (bs, n_dec_seq, d_hidn), (bs, n_dec_seq, n_dec_seq), (bs, n_dec_seq, n_enc_seq)
            dec_outputs, self_attn_prob, dec_enc_attn_prob = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            self_attn_probs.append(self_attn_prob)
            dec_enc_attn_probs.append(dec_enc_attn_prob)
        # (bs, n_dec_seq, d_hidn), [(bs, n_dec_seq, n_dec_seq)], [(bs, n_dec_seq, n_enc_seq)]S
        return dec_outputs, self_attn_probs, dec_enc_attn_probs

In [0]:
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.encoder = Encoder(self.config)
        self.decoder = Decoder(self.config)
    
    def forward(self, enc_inputs, dec_inputs):
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        enc_outputs, enc_self_attn_probs = self.encoder(enc_inputs)
        # (bs, n_seq, d_hidn), [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        dec_outputs, dec_self_attn_probs, dec_enc_attn_probs = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        # (bs, n_dec_seq, n_dec_vocab), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        return dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs


In [93]:
# prepare_train(vocab, "<path of data>/ratings_train.txt", "<path of data>/ratings_train.json")
# prepare_train(vocab, "<path of data>/ratings_test.txt", "<path of data>/ratings_test.json")
!ls

attention_from_master_				      naver_news_csv.py
kowiki						      ratings_test.txt
kowiki-latest-pages-meta-current.xml.bz2v6ldi9cr.tmp  ratings_train.txt
kowiki.model					      README.md
kowiki.py					      stackoverflow.py
kowiki.vocab					      WikiExtractor.py


In [94]:
Transformer(config)

Transformer(
  (encoder): Encoder(
    (enc_emb): Embedding(8007, 256)
    (pos_emb): Embedding(257, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): multi_head_attention(
          (W_Q): Linear(in_features=256, out_features=256, bias=True)
          (W_K): Linear(in_features=256, out_features=256, bias=True)
          (W_V): Linear(in_features=256, out_features=256, bias=True)
          (W_0): Linear(in_features=256, out_features=256, bias=True)
          (sdpa): scaled_dot_product_attention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (layer_norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (pos_ffn): PoswiseFeedForwardNet(
          (conv1): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
          (conv2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
        )
        (layer_norm2): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    

In [95]:
# 5/12 이것을 하나하나 뜯어보자...
class MovieClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = Transformer(self.config)
        self.projection = nn.Linear(self.config.d_hidn, self.config.n_output, bias=False)
    
    def forward(self, enc_inputs, dec_inputs):
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs = self.transformer(enc_inputs, dec_inputs)
        # (bs, d_hidn)
        dec_outputs, _ = torch.max(dec_outputs, dim=1)
        # (bs, n_output)
        logits = self.projection(dec_outputs)
        # (bs, n_output), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        return logits, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs
class MovieDataSet(torch.utils.data.Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels = []
        self.sentences = []

        line_cnt = 0
        with open(infile, "r") as f:
            for line in f:
                line_cnt += 1

        with open(infile, "r") as f:
            for i, line in enumerate(tqdm(f, total=line_cnt, desc=f"Loading {infile}", unit=" lines")):
                data = json.loads(line)
                self.labels.append(data["label"])
                self.sentences.append([vocab.piece_to_id(p) for p in data["doc"]])
    
    def __len__(self):
        assert len(self.labels) == len(self.sentences)
        return len(self.labels)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor([self.vocab.piece_to_id("[BOS]")]))
def movie_collate_fn(inputs):
    labels, enc_inputs, dec_inputs = list(zip(*inputs))

    enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True, padding_value=0)
    dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels, dim=0),
        enc_inputs,
        dec_inputs,
    ]
    return batch
batch_size = 128
train_dataset = MovieDataSet(vocab, "./ratings_train.json")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=movie_collate_fn)
test_dataset = MovieDataSet(vocab, "./ratings_test.json")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=movie_collate_fn)


FileNotFoundError: ignored

In [0]:
""" 모델 epoch 평가 """
def eval_epoch(config, model, data_loader):
    matchs = []
    model.eval()

    n_word_total = 0
    n_correct_total = 0
    with tqdm(total=len(data_loader), desc=f"Valid") as pbar:
        for i, value in enumerate(data_loader):
            labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

            outputs = model(enc_inputs, dec_inputs)
            logits = outputs[0]
            _, indices = logits.max(1)

            match = torch.eq(indices, labels).detach()
            matchs.extend(match.cpu())
            accuracy = np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

            pbar.update(1)
            pbar.set_postfix_str(f"Acc: {accuracy:.3f}")
    return np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

In [0]:
def train_epoch(config, epoch, model, criterion, optimizer, train_loader):
    losses = []
    model.train()

    with tqdm(total=len(train_loader), desc=f"Train {epoch}") as pbar:
        for i, value in enumerate(train_loader):
            labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

            optimizer.zero_grad()
            outputs = model(enc_inputs, dec_inputs)
            logits = outputs[0]

            loss = criterion(logits, labels)
            loss_val = loss.item()
            losses.append(loss_val)

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss_val:.3f} ({np.mean(losses):.3f})")
    return np.mean(losses)

In [0]:
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.n_output = 2
print(config)

learning_rate = 5e-5
n_epoch = 10

In [0]:
model = MovieClassification(config)
model.to(config.device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses, scores = [], []
for epoch in range(n_epoch):
    loss = train_epoch(config, epoch, model, criterion, optimizer, train_loader)
    score = eval_epoch(config, model, test_loader)

    losses.append(loss)
    scores.append(score)